## Creating MCP Server

In [1]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers together"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """Greet a user by name"""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

Writing mcp_server.py


## Creating client to test locally

In [2]:
%%writefile my_mcp_client.py
import asyncio
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

Writing my_mcp_client.py


## Local testin

In [6]:
# python mcp_server.py in CLI
!python my_mcp_client.py

Available tools:
  - add_numbers: Add two numbers together
  - multiply_numbers: Multiply two numbers together
  - greet_user: Greet a user by name


## Setting up Amazon Cognito for Authentication

In [12]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# utils_dir = os.path.join(current_dir, '..')
# utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
# sys.path.insert(0, utils_dir)
sys.path.insert(0, current_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

sys.path[0]: /Users/ykhvainitski/repos/bedrock-strands
Setting up Amazon Cognito user pool...
Pool id: us-east-1_awjGJtQxs
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_awjGJtQxs/.well-known/openid-configuration
Client ID: 6en4llrv22q7cdsqnat97d4175
Bearer Token: eyJraWQiOiI2R2M5WjVaTzJ6Q0pFTlp1UiszSjNRUFI2YWFlQk5OZUJLMzMwY0RZV2xVPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiI1NDY4MTRjOC1jMGIxLTcwOTYtNWM5Yy04YjZjMjIwYjUzYWYiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV9hd2pHSnRReHMiLCJjbGllbnRfaWQiOiI2ZW40bGxydjIycTdjZHNxbmF0OTdkNDE3NSIsIm9yaWdpbl9qdGkiOiIwMjM5NGU0Yi1hNTg3LTQxNmItYTBhNi0xMDUwYTZmNWFiYjEiLCJldmVudF9pZCI6ImI4MjFkOGYyLWNiYjktNDcwNi05NjE0LTlkYWE5YmY5MmIwZSIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NTYzMDk1MjYsImV4cCI6MTc1NjMxMzEyNiwiaWF0IjoxNzU2MzA5NTI2LCJqdGkiOiIwMDVkNmRkOS03YzE5LTQwMWMtYjRjYy1kYzJlNzgyYmNkMTciLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.E7ObTVHF8cHJkqrw35cug

## Configuring AgentCore Runtime Deployment

In [13]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp_server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name="mcp_server_agentcore"
)
print("Configuration completed ✓")

Entrypoint parsed: file=/Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial_mcp/mcp_server.py, bedrock_agentcore_name=mcp_server
Configuring BedrockAgentCore agent: mcp_server_agentcore


Using AWS region: us-east-1
All required files found ✓
Configuring AgentCore Runtime...


Generated .dockerignore
Generated Dockerfile: /Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial_mcp/Dockerfile
Generated .dockerignore: /Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial_mcp/.dockerignore
Setting 'mcp_server_agentcore' as default agent
Bedrock AgentCore configured: /Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial_mcp/.bedrock_agentcore.yaml


Configuration completed ✓
